# 40 Eridani A Mission Analysis
This notebook loads UQ and perf artifacts generated by the pipeline and summarizes feasibility and energy variability.

In [ ]:
# Imports
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Load artifacts (paths relative to repo root)
uq_summary_path = Path('uq_summary.json')
uq_records_path = Path('uq_records.jsonl')
perf_summary_path = Path('perf_aggregate.json')

uq_summary = json.loads(uq_summary_path.read_text()) if uq_summary_path.exists() else {}
records = []
if uq_records_path.exists():
    with open(uq_records_path) as f:
        for line in f:
            line = line.strip()
            if line:
                records.append(json.loads(line))
perf_summary = json.loads(perf_summary_path.read_text()) if perf_summary_path.exists() else {}

len(records), uq_summary.get('feasible_fraction'), uq_summary.get('energy_cv')

In [ ]:
# Compute basic metrics and plot energy distribution
import math
energies = [r['planned_energy'] for r in records if 'planned_energy' in r]
if energies:
    mean_e = sum(energies)/len(energies)
    var_e = sum((e-mean_e)**2 for e in energies)/max(1, len(energies)-1)
    std_e = math.sqrt(var_e)
    cv_e = std_e/mean_e if mean_e>0 else float('nan')
    print({'energy_mean': mean_e, 'energy_std': std_e, 'energy_cv': cv_e, 'feasible_fraction': uq_summary.get('feasible_fraction')})
    plt.figure(figsize=(6,4))
    plt.hist(energies, bins=15, color='#4C78A8', edgecolor='white')
    plt.title('Planned Energy Distribution')
    plt.xlabel('Energy (J)')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig('40eridani_energy.png', dpi=150)
else:
    print('No records found.')

In [ ]:
# Feasibility over samples (binary feasible field)
import numpy as np
feas = [1 if r.get('feasible') else 0 for r in records]
if feas:
    n = len(feas)
    win = max(1, n // 10)
    roll = np.convolve(feas, np.ones(win)/win, mode='same')
    plt.figure(figsize=(6,4))
    plt.plot(roll, color='#F58518')
    plt.ylim(0,1)
    plt.title('Feasible Fraction (rolling)')
    plt.xlabel('Sample Index')
    plt.ylabel('Feasible Fraction')
    plt.tight_layout()
    plt.savefig('40eridani_feasibility.png', dpi=150)
else:
    print('No feasibility data.')